Experimento 1 - KNN Imputer, 3sigma, min max normalization, knn classifier

In [31]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
from ucimlrepo import fetch_ucirepo

In [33]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [34]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [38]:
import random

#Introducing Missing values (5%)

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      df.loc[jndex,i] = np.NaN



In [39]:
df.isna().sum()

Area               294
Perimeter          281
MajorAxisLength    273
MinorAxisLength    251
AspectRatio        276
Eccentricity       252
ConvexArea         305
EquivDiameter      268
Extent             254
Solidity           257
Roundness          255
Compactness        251
ShapeFactor1       279
ShapeFactor2       272
ShapeFactor3       286
ShapeFactor4       270
dtype: int64

In [36]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [ ]:
#Imputing Missing Values with KNN Imputer
from sklearn.impute import KNNImputer as knni

imputer = knni(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
df_imputed

ModuleNotFoundError: No module named 'sklearn'

In [37]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

NameError: name 'df_imputed' is not defined

In [ ]:
#Outlier Removal with 3sigma
from scipy import stats


print(f'Pre Outlier Shape: {df_imputed.shape}')
df_no_outliers = df_imputed.copy()

print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      upper = class_unique[feature].mean() + (3 * class_unique[feature].std())
      lower = class_unique[feature].mean() - (3 * class_unique[feature].std())
      excluded = pd.Series(class_unique[class_unique[feature] < lower].index)
      #print(excluded.values)
      df_no_outliers.drop(excluded.values, inplace = True, errors='ignore')

print(df_no_outliers.groupby('Class').count())
print(f'Pos Outlier Shape: {df_no_outliers.shape}')


Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
       Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRatio  \
Class                                                                   
0      1263       1263             1263             1263         1263   
1       480        480              480              480          480   
2      1579       1579             1579             1579         1579   
3      3408       3408             3408             3408         3408   
4      1801       1801             1801             1801         1801   
5      1925       1925             1925             1925         1925   
6      2533       2533             2533             2533         2533   

       Eccentricity  ConvexArea  EquivDiameter  Extent  Solidity  Roundness  \
Class                                                                         
0              1263        1263           1263    1263      1263       1263   
1               480         480            480     480    

In [ ]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1263  1263      1263            1263            1263        1263   
BOMBAY     480   480       480             480             480         480   
CALI      1579  1579      1579            1579            1579        1579   
DERMASON  3408  3408      3408            3408            3408        3408   
HOROZ     1801  1801      1801            1801            1801        1801   
SEKER     1925  1925      1925            1925            1925        1925   
SIRA      2533  2533      2533            2533            2533        2533   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [ ]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28734.0,638.0180,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.996193,SEKER
1,29380.0,624.1100,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.754623,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
2,30279.0,634.9270,212.560556,181.510182,1.171067,0.520401,30600.0,196.347702,0.775688,0.989510,0.943852,0.923726,0.007020,0.003153,0.853270,0.999236,SEKER
3,30477.0,668.7638,211.050155,184.039050,1.146768,0.489478,30970.0,196.988633,0.762402,0.984081,0.853080,0.933374,0.006925,0.003242,0.871186,0.999049,SEKER
4,30519.0,629.7270,212.996755,182.737204,1.165591,0.513760,30847.0,197.124320,0.770682,0.989367,0.967109,0.925480,0.006979,0.003158,0.856514,0.998345,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12984,42097.0,759.6960,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.988100,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
12985,42101.0,757.4990,281.576392,190.713136,1.476439,0.762798,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
12986,42139.0,759.3210,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
12987,42147.0,763.7790,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.804446,0.006724,0.001852,0.668237,0.995222,DERMASON


In [ ]:
#Normalization with min max
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,0.034774,0.080647,0.026167,0.160782,0.034711,0.068251,0.034614,0.073829,0.735504,0.632906,0.683292,0.935127,0.590062,1.000000,0.946861,0.867762,SEKER
1,0.037574,0.070686,0.048449,0.140205,0.116077,0.349735,0.036805,0.079116,0.641111,0.806121,0.869115,0.800547,0.627322,0.828005,0.787671,0.975101,SEKER
2,0.041470,0.078433,0.047968,0.157079,0.088091,0.270803,0.040654,0.086379,0.708871,0.804267,0.856900,0.845177,0.595878,0.863027,0.839608,0.981439,SEKER
3,0.042328,0.102667,0.045232,0.164728,0.070494,0.213136,0.042218,0.087964,0.666131,0.598657,0.579546,0.873974,0.582510,0.892778,0.873570,0.974450,SEKER
4,0.042510,0.074709,0.048758,0.160791,0.084125,0.258418,0.041698,0.088299,0.692766,0.798853,0.927963,0.850414,0.590131,0.864862,0.845758,0.948139,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12984,0.092692,0.167792,0.185924,0.170492,0.364480,0.726947,0.091018,0.173345,0.512286,0.750878,0.773641,0.481432,0.573181,0.395047,0.441003,0.949660,DERMASON
12985,0.092709,0.166218,0.172982,0.184915,0.309234,0.722837,0.090959,0.173373,0.786890,0.851303,0.790178,0.542286,0.549250,0.440635,0.503767,0.943435,DERMASON
12986,0.092874,0.167523,0.172916,0.186351,0.306440,0.669254,0.091276,0.173631,0.561689,0.818998,0.779205,0.543711,0.548281,0.441448,0.505256,0.889205,DERMASON
12987,0.092909,0.170716,0.176254,0.183592,0.318566,0.682289,0.091690,0.173685,0.482741,0.739982,0.747069,0.489135,0.554244,0.429363,0.488864,0.831482,DERMASON


In [ ]:
#Classification with Knn-Classification

from sklearn.neighbors import KNeighborsClassifier as knnc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing


classifier = knnc(n_neighbors=10)
x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}

y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)


print('Experimento 1')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')

Experimento 1
Acurácia Média: 90.08%
Precisão Média: 93.13%
Revocação Média: 91.66%
F1-Score Médio: 91.53%
